# Evaluation Part II
一些 LLM 回覆錯誤的範例

### 環境設定
以下是環境的準備， 執行前請先在 `.env` 檔案內填入您自己的 openai api key。

In [1]:
import os
import openai
import sys
sys.path.append('../..')
import utils
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0, max_tokens=16000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

### Run through the end-to-end system to answer the user query

In [23]:
customer_msg = f"""
可以幫我介紹關於 smartX ProPhone 手機和 fotosnap 單眼相機的資訊嗎？"""

products_by_category = utils.get_products_from_query(customer_msg)
category_and_product_list = utils.read_string_to_list(products_by_category)
product_info = utils.get_mentioned_product_info(category_and_product_list)
assistant_answer = utils.answer_user_msg(user_msg=customer_msg,
                                                   product_info=product_info)

In [24]:
print(assistant_answer) 

當然可以！我們有兩款產品可以介紹給您。

首先是 SmartX ProPhone 手機。這款手機具有6.1吋的顯示器和128GB的儲存空間，讓您可以盡情享受多媒體內容。它還配備了12MP的雙鏡頭，讓您拍攝出色的照片和視頻。此外，它還支援5G網路，讓您在網絡上瀏覽和下載速度更快。SmartX ProPhone 手機的價格為899.99元，並提供一年的保修。

另一款產品是 FotoSnap 單眼相機。這款相機配備了24.2MP的感光元件，可以捕捉出色的照片和1080p的高清視頻。它還擁有一個3吋的LCD螢幕，方便您即時查看拍攝的內容。最特別的是，它還支援可更換鏡頭，讓您根據不同的拍攝需求進行調整。FotoSnap 單眼相機的價格為599.99元，同樣提供一年的保修。

請問您對這兩款產品有什麼具體的問題嗎？


### 根據提取的產品資訊，以 Rubic 評分標準評估 LLM 模型回答給使用這的回覆

In [5]:
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [6]:
def eval_with_rubric(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    system_message = """\
    你是一個助理，透過檢視客服人員用來產生其回應的上下文， \
    評估客服人員回答使用者問題的表現。
    """

    user_message = f"""\
    你正在根據代理人用來回答問題的上下文，對提交的答案進行評估。
    以下是數據：
    [數據開始]
    ************
    [問題]: {cust_msg}
    ************
    [上下文]: {context}
    ************
    [提交的答案]: {completion}
    ************
    [數據結束]

    比較提交答案的事實內容與上下文，忽略風格、文法或標點符號的任何差異。
    回答以下問題：
    - 助理的回答僅基於所提供的上下文嗎？（是或否）
    - 答案是否包含上下文中未提供的資訊？（是或否）
    - 回應與上下文之間是否有任何不符？（是或否）
    - 計算使用者問了多少個問題。（輸出一個數字）
    - 對於使用者提出的每個問題，是否有相對應的答案？
    問題1：（是或否）
    問題2：（是或否）
    ...
    問題N：（是或否）
    - 在問的問題數中，有多少個問題得到了答案的回應？（輸出一個數字）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [7]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

助理的回答僅基於所提供的上下文嗎？是
答案是否包含上下文中未提供的資訊？否
回應與上下文之間是否有任何不符？否
計算使用者問了多少個問題。1
對於使用者提出的每個問題，是否有相對應的答案？
問題1：是
在問的問題數中，有多少個問題得到了答案的回應？1


### Evaluate the LLM's answer to the user based on an "ideal" / "expert" (human generated) answer.

In [25]:
test_set_ideal = {
    'customer_msg': """ \
    告訴我有關 smartX ProPhone 的手機和 fotosnap 相機的資訊， \
    就是那款數位單眼相機的。另外你們有什麼電視或者電視相關的產品？""",
    
    'ideal_answer':""" \
    當然可以！SmartX ProPhone 是一款功能強大的智慧型手機，具有先進的相機功能。 \
    例如，它擁有一個 1200 萬像素的雙鏡頭、5G 無線和 128GB 的儲存空間。 \
    它還有一個 6.1 吋的顯示器。價格是 $899.99。

    FotoSnap 數位單眼相機非常適合捕捉令人驚嘆的照片和影片。 \ 
    一些功能包括 1080p 的影片，3 吋液晶螢幕，2420 萬像素的感光元件，以及可更換鏡頭。 \
    價格是 $599.99。

    你對這些產品有什麼其他的問題嗎？
    或者有其他我可以幫你解答的問題嗎？
    """
}

In [9]:
test_set_ideal2 = {
    'customer_msg': """ \
    告訴我有關 smartX ProPhone 的手機和 fotosnap 相機的資訊， \
    就是那款數位單眼相機的。另外你們有什麼電視或者電視相關的產品？""",
    
    'ideal_answer':""" \
    當然可以！SmartX ProPhone 是一款功能強大的智慧型手機，具有先進的相機功能。 \

    FotoSnap 數位單眼相機非常適合捕捉令人驚嘆的照片和影片。 \ 

    對於電視和電視相關產品，我們則提供了 3 種電視產品。

    所有的電視都提供 HDR 和智慧型電視。

    像是 CineView 4K 液晶電視， CineView 8K 液晶電視還有CineView OLED 電視\
    
    我們也提供 2 種家庭劇院產品，像是SoundMax 家庭劇院， SoundMax 音箱等

    你對這些產品有什麼其他的問題嗎？
    或者有其他我可以幫你解答的問題嗎？
    """
}

### Check if the LLM's response agrees with or disagrees with the expert answer
This evaluation prompt is from the [OpenAI evals](https://github.com/openai/evals/blob/main/evals/registry/modelgraded/fact.yaml) project.

[BLEU score](https://en.wikipedia.org/wiki/BLEU): another way to evaluate whether two pieces of text are similar or not.

In [5]:
def eval_vs_ideal(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    print(f'cust_msg: {cust_msg}')
    print(f'ideal: {ideal}')
    print(f'completion: {completion}')

    system_message = """\
    你是一個助手，透過比較回答和理想（專家）的回覆來評估客服代理如何回答用戶問題。\
    你只能輸出一個字母，除此之外別無他物。
    """

    user_message = f"""\
    你正在對給定問題的答案比較提交的答案和專家回覆。以下是數據：
    [數據開始]
    ************
    [問題]: {cust_msg}
    ************
    [專家]: {ideal}
    ************
    [回答]: {completion}
    ************
    [數據結束]

    將提交的回答的實際內容與專家的回覆進行比較。 \
    忽略風格、語法或標點的差異。提交的回答可能是專家答案的子集或超集， \
    或者可能與專家答案有衝突。確定哪種情況適用。選擇以下選項之一來回答問題：
    (A) 提交的回答是專家答案的子集，並且與專家答案完全一致。
    (B) 提交的回答是專家答案的超集，並且與專家答案完全一致。
    (C) 提交的回答包含與專家答案相同的所有細節。
    (D) 提交的回答和專家答案之間有不同意見。
    (E) 答案有所不同，但從實際性的角度看，這些差異並不重要。
    """ 

    #print(f'user_msg: {user_message}')

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [6]:
def eval_vs_ideal2(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    print(f'cust_msg: {cust_msg}')
    print(f'ideal: {ideal}')
    print(f'completion: {completion}')
    
    system_message = """\
    你是一個評分的助手，請透過實際回答與專家回答的內容比較來做評估。\
    評估結果你只能回答一個字母。
    """

    user_message= f"""\
    你正在針對原始問題來做實際回答以及專家回答間的差異評分，並且依照評分邏輯做為判斷邏輯。

    [評分資料開始]
    ************
    [原始問題]: {cust_msg}
    ************
    [專家回答]: {ideal}
    ************
    [實際回答]: {completion}
    ************
    [評分邏輯]:
    1. 將實際回答內容與專家的回答內容進行比較，忽略風格、語法、順序或標點的差異。
    2. 如果兩者回覆的內容，只是表達方式和詳細程度不同，請當作相同的回答。
    3. 實際回答可能是專家回答的子集或超集，或者可能與專家回答有衝突。
    ************
    [評分資料結束]
    
    請你確定下方哪種情況適用，並且只回覆 ABCDE 中的一個字元：
    (A) 實際回答是專家回答的子集。
    (B) 實際回答是專家回答的超集。
    (C) 實際回答和專家答案之間有不同意見。
    (D) 實際回答和專家答案相同
    (E) 答案有所不同，但從實際性的角度看，這些差異並不重要。

    """
    

    #print(f'user_msg: {user_message}')

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [7]:
def eval_vs_ideal_org(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    print(f'cust_msg: {cust_msg}')
    print(f'ideal: {ideal}')
    print(f'completion: {completion}')
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by comparing the response to the ideal (expert) response
    Output a single letter and nothing else. 
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Expert]: {ideal}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a subset of the expert answer and is fully consistent with it.
    (B) The submitted answer is a superset of the expert answer and is fully consistent with it.
    (C) The submitted answer contains all the same details as the expert answer.
    (D) There is a disagreement between the submitted answer and the expert answer.
    (E) The answers differ, but these differences don't matter from the perspective of factuality.
  choice_strings: ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

### eval_vs_ideal 的測試

In [26]:
ideal_answers = [
    """
    南洋地區（東南亞）擁有許多獨特的水果，以下是一些常見的例子：
    芒果：這個地區的芒果口感甜美，種類多樣。
    榴槤：被譽為"果王"，榴槤的口感獨特，味道濃郁。
    荔枝：具有甜美的味道和多汁的肉質，是夏季的理想水果。
    """,
    """
    芒果、荔枝、榴槤
    """
]

assist_msg = [
    """
    芒果、榴槤
    """, # 子集
    """
    南洋水果有： 芒果、榴槤、荔枝。
    """, # 完全相同
    """
    芒果、榴槤、荔枝、龍眼、香蕉、椰子、番荔枝、石榴、菠蘿、火龍果、木瓜
    """, # 超集
    """
    南洋有很多水果，例如：
    芒果: 甜美。
    榴槤： 果王
    荔枝：甜美多汁。
    """, # 含義相同
    """
    東南亞的代表性水果有： 蘋果、梨子、水蜜桃。
    """
]

test_set = {} 
test_set['customer_msg'] = """
南洋水果有哪些?
"""

test_set['ideal_answer'] = ideal_answers[0]

#eval_vs_ideal_org(test_set, assist_msg[1])
#eval_vs_ideal(test_set, assist_msg[1])
#eval_vs_ideal2(test_set, assist_msg[2])
eval_vs_ideal2(test_set_ideal, assistant_answer)


cust_msg:      告訴我有關 smartX ProPhone 的手機和 fotosnap 相機的資訊，     就是那款數位單眼相機的。另外你們有什麼電視或者電視相關的產品？
ideal:      當然可以！SmartX ProPhone 是一款功能強大的智慧型手機，具有先進的相機功能。     例如，它擁有一個 1200 萬像素的雙鏡頭、5G 無線和 128GB 的儲存空間。     它還有一個 6.1 吋的顯示器。價格是 $899.99。

    FotoSnap 數位單眼相機非常適合捕捉令人驚嘆的照片和影片。 \ 
    一些功能包括 1080p 的影片，3 吋液晶螢幕，2420 萬像素的感光元件，以及可更換鏡頭。     價格是 $599.99。

    你對這些產品有什麼其他的問題嗎？
    或者有其他我可以幫你解答的問題嗎？
    
completion: 當然可以！我們有兩款產品可以介紹給您。

首先是 SmartX ProPhone 手機。這款手機具有6.1吋的顯示器和128GB的儲存空間，讓您可以盡情享受多媒體內容。它還配備了12MP的雙鏡頭，讓您拍攝出色的照片和視頻。此外，它還支援5G網路，讓您在網絡上瀏覽和下載速度更快。SmartX ProPhone 手機的價格為899.99元，並提供一年的保修。

另一款產品是 FotoSnap 單眼相機。這款相機配備了24.2MP的感光元件，可以捕捉出色的照片和1080p的高清視頻。它還擁有一個3吋的LCD螢幕，方便您即時查看拍攝的內容。最特別的是，它還支援可更換鏡頭，讓您根據不同的拍攝需求進行調整。FotoSnap 單眼相機的價格為599.99元，同樣提供一年的保修。

請問您對這兩款產品有什麼具體的問題嗎？


'B'

In [28]:
print(assistant_answer)

關於SmartX ProPhone手機，它是一款功能強大並具有先進照相功能的智慧手機。它擁有6.1吋顯示器、128GB儲存空間、12MP雙鏡頭和5G網路。價格為899.99美元，保固期為一年。

至於FotoSnap數位單眼相機，它是一款多功能的單眼相機，能夠捕捉精彩的照片和視頻。它擁有24.2MP感光元件、1080p影片、3吋LCD和可更換鏡頭。價格為599.99美元，保固期為一年。

此外，我們還有其他電視和電視相關的產品。例如CineView 4K液晶電視，它擁有55吋顯示器、4K解析度、HDR和智慧電視功能。價格為599.99美元，保固期為兩年。還有SoundMax家庭劇院系統，它擁有5.1聲道、1000W輸出、無線重低音和藍牙功能。價格為399.99美元，保固期為一年。還有CineView 8K液晶電視和CineView OLED電視等其他選擇。

請問您對這些產品有什麼具體的問題嗎？


In [31]:
eval_vs_ideal(test_set_ideal2, assistant_answer)

cust_msg:      告訴我有關 smartX ProPhone 的手機和 fotosnap 相機的資訊，     就是那款數位單眼相機的。另外你們有什麼電視或者電視相關的產品？
ideal:      當然可以！SmartX ProPhone 是一款功能強大的智慧型手機，具有先進的相機功能。 
    FotoSnap 數位單眼相機非常適合捕捉令人驚嘆的照片和影片。 \ 

    對於電視和電視相關產品，我們則提供了 3 種電視產品。

    所有的電視都提供 HDR 和智慧型電視。

    像是 CineView 4K 液晶電視， CineView 8K 液晶電視還有CineView OLED 電視    
    我們也提供 2 種家庭劇院產品，像是SoundMax 家庭劇院， SoundMax 音箱等

    你對這些產品有什麼其他的問題嗎？
    或者有其他我可以幫你解答的問題嗎？
    
completion: 關於SmartX ProPhone手機，它是一款功能強大並具有先進照相功能的智慧手機。它擁有6.1吋顯示器、128GB儲存空間、12MP雙鏡頭和5G網路。價格為899.99美元，保固期為一年。

至於FotoSnap數位單眼相機，它是一款多功能的單眼相機，能夠捕捉精彩的照片和視頻。它擁有24.2MP感光元件、1080p影片、3吋LCD和可更換鏡頭。價格為599.99美元，保固期為一年。

此外，我們還有其他電視和電視相關的產品。例如CineView 4K液晶電視，它擁有55吋顯示器、4K解析度、HDR和智慧電視功能。價格為599.99美元，保固期為兩年。還有SoundMax家庭劇院系統，它擁有5.1聲道、1000W輸出、無線重低音和藍牙功能。價格為399.99美元，保固期為一年。還有CineView 8K液晶電視和CineView OLED電視等其他選擇。

請問您對這些產品有什麼具體的問題嗎？


'D'

In [14]:
assistant_answer_2 = "life is like a box of chocolates"

In [15]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)

'D'